In [1]:
import polars as pl

import nwec.utility_reporting.arrearages
import nwec.utils.excel
from nwec.constants import CLEAN_UTILITY_DATA, RAW_UTILITY_DATA

YEAR = 2024
NUM_MONTHS = 3
COLS_PER_MONTH = 4
SHEET_SEARCH_STRING = "Past Due Balances"
ARREARAGE_SEARCH_STRING = "Past-due balances by customer class and number of days"
KLI_SEARCH_STRING = "past-due balances for known low-income"
spreadsheet = RAW_UTILITY_DATA / f"pse/pse_{YEAR}.xlsx"

In [2]:
sheet_index = nwec.utils.excel.get_sheet_index_from_name(spreadsheet, SHEET_SEARCH_STRING)
df = pl.read_excel(spreadsheet, sheet_id=sheet_index, has_header=False)
arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(
    df, NUM_MONTHS, COLS_PER_MONTH, ARREARAGE_SEARCH_STRING
)
kli_arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(df, NUM_MONTHS, COLS_PER_MONTH, KLI_SEARCH_STRING)

In [3]:
arrearages

Zip Code,Customer Class,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21
str,str,str,str,str,str,str,str,str,str,str,str,str,str
null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""98943""","""Residential""","""12138.76""","""11900.35""","""18503.08""","""56740.4""","""7944.38""","""7671.38""","""22423.0899999999""","""51866.6999999999""","""7096.4""","""6513.43""","""26313.9699999999""","""51572.54"""
"""98946""","""Commercial / Industrial""","""3994.65""","""3404.4""","""74532.03""","""98171.38""","""2964.63""","""3520.58""","""77519.62""","""94833.4""","""2272.54""","""2616.82""","""80497.05""","""98328.74"""
"""98946""","""Residential""","""6265.95""","""6603.97""","""16860.53""","""38514.03""","""4864.08""","""5016.91""","""20412.8299999999""","""36553.7""","""4279.46""","""4541.77""","""22540.0699999999""","""39278.38"""


# Arrearages

In [4]:
date_to_zip_offset = 0
source_date_format = "%Y-%m-%d %H:%M:%S"

arrearages = nwec.utility_reporting.arrearages.combine_arrearage_year_vintage_cols(
    arrearages, NUM_MONTHS, COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [5]:
arrearages.sum()

Zip Code,04 2024 31 - 60 Days,04 2024 61 - 90 Days,04 2024 91+ Days,04 2024 Total Arrearages,05 2024 31 - 60 Days,05 2024 61 - 90 Days,05 2024 91+ Days,05 2024 Total Arrearages,06 2024 31 - 60 Days,06 2024 61 - 90 Days,06 2024 91+ Days,06 2024 Total Arrearages
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
22404931,2.2155e7,1.8843e7,6.6271e7,1.4212e8,2.1600e7,1.5369e7,7.1822e7,1.3657e8,1.7212e7,1.6436e7,7.5010e7,1.3435e8


# KLI Arrearages

In [6]:
date_to_zip_offset = 0
source_date_format = "%Y-%m-%d %H:%M:%S"

kli_arrearages = nwec.utility_reporting.arrearages.combine_arrearage_year_vintage_cols(
    kli_arrearages, NUM_MONTHS, COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [7]:
kli_arrearages.sum()

Zip Code,04 2024 31 - 60 Days,04 2024 61 - 90 Days,04 2024 91+ Days,04 2024 Total Arrearages,05 2024 31 - 60 Days,05 2024 61 - 90 Days,05 2024 91+ Days,05 2024 Total Arrearages,06 2024 31 - 60 Days,06 2024 61 - 90 Days,06 2024 91+ Days,06 2024 Total Arrearages
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
21913360,4.3267e6,4.1498e6,1.3362e7,2.6811e7,3.8740e6,3.1100e6,1.3463e7,2.4092e7,2.8849e6,3.1755e6,1.3603e7,2.2751e7


# Save Output

In [8]:
arrearages = nwec.utility_reporting.arrearages.normalize_arrearage_cols(arrearages, NUM_MONTHS)
arrearages = arrearages.with_columns(pl.lit("PSE").alias("Utility"))
arrearages = arrearages.with_columns(pl.lit("Residential").alias("Customer Class"))

kli_arrearages = nwec.utility_reporting.arrearages.normalize_arrearage_cols(kli_arrearages, NUM_MONTHS)
kli_arrearages = kli_arrearages.with_columns(pl.lit("PSE").alias("Utility"))
kli_arrearages = kli_arrearages.with_columns(pl.lit("KLI").alias("Customer Class"))

pl.concat([arrearages, kli_arrearages]).write_ipc(CLEAN_UTILITY_DATA / "pse.arrow")